In [1]:
import h5py
import re
import torch
import torch.nn as nn
from torch.legacy.nn import Reshape
import graphviz
import torch.nn.functional as F
from torch.autograd import Variable
#from visualize import make_dot
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as utils
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imresize, imread, imshow
import time
import logging
from math import log,sqrt

/home/suman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [45]:
 vgg16 = models.vgg16(pretrained=True)
#vgg16 = models.vgg16()
# print(vgg16)

In [44]:
# vgg16.classifier = nn.Sequential(*list(vgg16.classifier.children())[:-1])
# vgg16 = nn.Sequential(*list(vgg16.children())[:-1])
# print(vgg16)

VGG (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU (inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU (inplace)
    (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU (inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU (inplace)
    (9): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU (inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU (inplace)
    (16): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), pa

In [ ]:
#vgg16 = nn.Sequential(*list(vgg16.children())[:-1])

In [46]:
def InitializeWeights(mod):
    for m in mod.modules():
        if isinstance(m,nn.Conv2d):
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            #print m.weight.size(), m.out_channels, m.in_channels
            m.weight.data.normal_(0,sqrt(2./n))
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.bias.data.zero_()
    return mod

In [47]:
# conv1 = nn.Sequential(nn.BatchNorm2d(512),nn.ReLU(),nn.Conv2d(512,1024,1))
# conv1 = InitializeWeights(conv1)
# conv2 = nn.Sequential(nn.BatchNorm2d(1024),nn.ReLU(),nn.Conv2d(1024,128,5))
# conv2 = InitializeWeights(conv2)
# conv3 = nn.Sequential(nn.BatchNorm2d(128),nn.ReLU(),nn.Conv2d(128,16,1))
# conv3 = InitializeWeights(conv3)
fc1= InitializeWeights(nn.Sequential(nn.ReLU(),nn.Linear(4096, 1)))

In [52]:
class MyModel4(nn.Module):
    def __init__(self, pretrained_model):
        super(MyModel4, self).__init__()
        self.pretrained_model1 = nn.Sequential(*list(pretrained_model.children())[:-1])
        self.pretrained_model2 = nn.Sequential(*list(pretrained_model.classifier.children())[:-1])
        self.fc1 =fc1

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    ##########
    def shuffled_features(self,x):
        num_features = 25088 #vgg16 after removing classifier
        size=x.size()[1:]
        shuffled_size = np.random.permutation(x.size()[1])
        yy = x[:,shuffled_size[0:num_features]]
        return yy
    
        
    def forward(self, x):
        x = self.pretrained_model1(x) 
#         x = self.conv1(x)
#         x = self.conv2(x)
#         x = self.conv3(x)
        x = x.view(-1, self.num_flat_features(x))
        x = self.shuffled_features(x)
        x = self.pretrained_model2(x)
        x = self.fc1(x)
#         #x = x.view(-1, self.num_flat_features(x))
#         #print(type(x))
        return x
        
    

In [53]:
net = MyModel4(vgg16)

In [54]:
net

MyModel4 (
  (pretrained_model1): Sequential (
    (0): Sequential (
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU (inplace)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU (inplace)
      (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU (inplace)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU (inplace)
      (9): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU (inplace)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU (inplace)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU (inplace)
      (16): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1

In [55]:
input=Variable(torch.randn(1,3,180,320))
output=net(input)
print(output.size())

torch.Size([1, 1])


In [24]:
input = Variable(torch.randn(5, 3, 180, 320))
out = net(input)
print(out.size())

torch.Size([5, 25088])


In [ ]:
sum1 = 0
        
print("Number of layers ---> ",len(list(vgg16.parameters())))
for params in vgg16.parameters():
    if params.requires_grad == True:
        sum1 += params.numel()
    
print("Total number of parameters ---> ",sum1)

In [ ]:
file = h5py.File('./DATASET/CODE/NewTrainData.h5')
xtrainT = torch.from_numpy(np.array(file['xtrain'],dtype=np.float32)).float()
ytrainT = torch.from_numpy(np.array(file['ytrain'],dtype=np.float32)).float()
#xtrain = np.array(file['xtrain'],dtype=np.float32)
#ytrain = np.array(file['ytrain'],dtype=np.float32)

In [ ]:
file = h5py.File('./DATASET/CODE/NewTestData_19400.h5')
xtestT = torch.from_numpy(np.array(file['xtest'],dtype=np.float32)).float()
ytestT = torch.from_numpy(np.array(file['ytest'],dtype=np.float32)).float()
#xtest = np.array(file['xtest'],dtype=np.float32)
#ytest = np.array(file['ytest'],dtype=np.float32)

In [ ]:
def batch_rgb_to_bgr(batch):
    #print(batch.size())
    (r, g, b) = torch.chunk(batch, 3, 1)
    #print(r.size())
    batch1 = torch.cat((b, g, r),1)
    #print(batch1.size())
    return batch1

In [ ]:
xtrainT = batch_rgb_to_bgr(xtrainT)
xtestT = batch_rgb_to_bgr(xtestT)
print(xtrainT.size(), xtestT.size())

In [ ]:
xtrainT = torch.div(xtrainT,255.0)
xtestT = torch.div(xtestT,255.0)

In [ ]:
print(torch.min(xtrainT), torch.max(xtrainT), torch.min(xtestT), torch.max(xtestT))
print(xtrainT.size(), ytrainT.size(), xtestT.size(), ytestT.size())

In [ ]:
class Normalize(object):
    """
    Normalize an tensor image with mean and standard deviation.
    Given mean: (R, G, B) and std: (R, G, B),
    will normalize each channel of the torch.*Tensor, i.e.
    channel = (channel - mean) / std
    Args:
        mean (sequence): Sequence of means for R, G, B channels respecitvely.
        std (sequence): Sequence of standard deviations for R, G, B channels
            respecitvely.
    """

    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        # TODO: make efficient
        for t, m, s in zip(tensor, self.mean, self.std):
            t.sub_(m).div_(s)
        return tensor

In [ ]:
mn = [0.406,0.456,0.485]
sd = [0.225,0.224,0.229]
norm = Normalize(mn,sd)
xtrainT = norm(xtrainT)
xtestT = norm(xtestT)

In [ ]:
print(torch.min(xtrainT), torch.max(xtrainT), torch.min(xtestT), torch.max(xtestT))

In [ ]:
##def train(model, loss, optimizer, x_val, y_val, validPixel, batch_sz):
def train(model, loss, optimizer, x_val, y_val,batch_size):
    x = Variable(x_val,requires_grad = False).cuda()
    y = Variable(y_val,requires_grad = False).cuda()
    
    optimizer.zero_grad()
    
    fx = model.forward(x)
    
    #print fx.data[0][0][64][87]
    #fx = model5.forward(Variable(xtest2[start:end], volatile=True).cuda())
    ##output = loss.forward(fx,y,validPixel,batch_sz)
    output = loss.forward(fx,y)
    #output = loss(fx, y)
    output.backward()
    
    optimizer.step()
    
    return output.data[0]

In [ ]:

class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
        
    def forward(self,inp, tar):
        #target is the ground truth value...
        #k = torch.mean(inp[:,0])
        '''
        if (k >= 1.48 and k <= 1.65):
            diff = torch.abs(tar[:,1]-inp[:,1])
            loss = torch.mean(torch.pow(diff,2))
        else:
        '''
        diff = torch.abs(tar[:,0]-inp[:,0]) #*(180/np.pi)
        loss = torch.mean(diff)
        #print(loss)
        return loss
        '''
        c1 = c.data[0] 
        temp = diff > c1
        check1 = torch.prod(temp)
        
        if check1 == 0:
            lossval = torch.mean(diff)
        else:
            temp4 = torch.pow(diff,2)
            d = torch.pow(c,2)
            temp4 = temp4.add(d.expand_as(temp4))
            lossval = torch.mean(temp4/(2*c))
        return lossval
        '''

In [ ]:
class HuberLoss(nn.Module):
    def __init__(self):
        super(HuberLoss, self).__init__()
        
    def forward(self,inp, tar):
        #target is the ground truth value...
        #k = torch.mean(inp[:,0])
        
        
        diff = torch.abs(tar[:,0]-inp[:,0]) #*(180/np.pi)
        
        if diff<=1:
            loss=0.5*(diff**2)
        else: 
            loss=0.5*1+(1*(diff-1)) 
        
        
        
        loss = torch.mean(diff)
        #print(loss)
        return loss
        

In [ ]:
class CosineLoss(nn.Module):
    def __init__(self):
        super(CosineLoss, self).__init__()
        
    def forward(self,inp, tar,batch_sz):
        alpha_t = tar[:,0]
        alpha_p = inp[:,0]
        #xt = torch.stack([torch.cos(alpha_t),torch.sin(alpha_t)])
        #xp = torch.stack([torch.cos(alpha_p),torch.sin(alpha_p)])
        #cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        #loss = cos(xt, xp)
        #return loss
        loss = Variable(torch.FloatTensor(batch_sz).zero_(), requires_grad=False).cuda()
        for i in range(batch_sz):          
            loss[i] = torch.cos(alpha_t[i:i+1]-alpha_p[i:i+1])
            
        lossval = 1.0-torch.mean(loss)    
        #print(lossval)
        return lossval

In [ ]:
-

In [ ]:
for params in optimizer.param_groups:
    print(params['lr'])

In [ ]:
net = net.cuda()
#net = torch.nn.DataParallel(net).cuda()
net.load_state_dict(torch.load("CustomLoss_new_35000_DISTANCE_109_epochs.pth"))


In [ ]:
#testing of the architecture...
num_batches = 0
#6 evenly divides the test batch size..
test_batch_size = 10
n_examples = 1980
#finalpred = Variable(torch.zeros((n_examples,3,120,160)))
finalpred = Variable(torch.zeros((n_examples,1)))
print("finalpred size is ---> ", finalpred.size())

num_batches = n_examples//test_batch_size
print("num of batches --->", num_batches)
for k in range(num_batches):
    start, end = k*test_batch_size, (k+1)*test_batch_size
    output = net.forward(Variable(xtestT[start:end], volatile=True).cuda())
    finalpred[start:end] = output

In [ ]:
data1 = finalpred.data.numpy()
print(data1.shape)

In [ ]:
dif = torch.abs(finalpred.data[:,0]-ytestT[:,0,2])
print(dif.size())
np.savetxt("diff.csv", dif.numpy(), delimiter=",")
MSEloss = torch.mean(torch.pow(dif,2))
ABSlossRad = torch.mean(dif)
ABSlossDeg = ABSlossRad*(180/np.pi)
print("MSEloss=="+str(MSEloss),"ABSlossRad=="+str(ABSlossRad),"ABSlossDeg"+str(ABSlossDeg))

In [ ]:
print(finalpred.size())
print(ytestT.size())

In [ ]:
n=452

print(finalpred.data[n,0]*(180/np.pi))
print(ytestT[n,0,2]*(180/np.pi))
print(ytestT[n,0,2]*(180/np.pi)-finalpred.data[n,0]*(180/np.pi))

In [ ]:
print(ABSlossRad*(180/np.pi))
print(MSEloss*(180/np.pi))

In [ ]:
#print(ytestT[:,0]*(180/np.pi))
#print(ytestT[:,0,2]*(180/np.pi))
a = ytestT[:,0,2]*(180/np.pi)
print(a.size())
np.savetxt("test.csv", a.numpy(), delimiter=",")

In [ ]:
#print(finalpred.data[:,0]*(180/np.pi))
print(finalpred.data[:,0]*(180/np.pi))
b = finalpred.data[:,0]*(180/np.pi)
c=torch.abs(ytestT[:,0,2]*(180/np.pi)- finalpred.data[:,0]*(180/np.pi))
np.savetxt("pred.csv", b.numpy(), delimiter=",")

np.savetxt("diff.csv", c.numpy(), delimiter=",")

In [ ]:
finalpred.data[1,0]

In [ ]:
import scipy
from scipy.misc import imresize, imread, imshow
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'none'
%matplotlib inline

fig = plt.figure()
fig.set_figheight(9)
fig.set_figwidth(9)
import cv2

test = cv2.imread("./Test_Net_image/4.JPG")
print(test.shape)
test = imresize(test,(180,320,3))
#imshow(test)
test = test.transpose(2,0,1)
test = np.reshape(test,(1,3,180,320))
test = test.astype(np.float32)
testPT = torch.from_numpy(test).float()
testPT = batch_rgb_to_bgr(testPT)
testPT = torch.div(testPT,255.0)
mn = [0.406,0.456,0.485]
sd = [0.225,0.224,0.229]
norm = Normalize(mn,sd)
testPT = norm(testPT)
'''
ind = 2000
testPT = xtestT[ind]
print("Actual angle==="+str(ytestT[ind,0]*(180/np.pi)), ytestT[ind,0])
testPT = testPT.view(1,3,180,320)
#'''
test_pred = net.forward(Variable(testPT, volatile=True).cuda())
print("Angle==="+str(test_pred.data[0,0]*(180/np.pi)), test_pred.data[0,0])
testx = testPT.numpy()
testx = np.reshape(testx,(3,180,320))
testx = testx.transpose(1,2,0)
testx = imresize(testx,(180,320,3))
#imshow(testx)
scipy.misc.imsave('test.png', testx)
a=fig.add_subplot(1,2,1)
imgplot = plt.imshow(testx)
a.set_title('Input')
a.axes.get_xaxis().set_visible(False)
a.axes.get_yaxis().set_visible(False)

In [ ]:
############__________Rough________##############
xx=Variable(torch.rand((5,1)))
print(xx)
#print(xx.size())

In [ ]:
######_______Rough______######
indices = np.random.permutation(xx.size())
print(indices)
#xx1 = xx.clone()
xx1=Variable(torch.zeros((5,1)))
#xx1 = torch.randperm(xx1)  
#print(xx.size())
xx=xx.view(-1,5)
print(xx.size())



ran=np.random.permutation(xx.size()[1])
print(ran.shape)

In [ ]:
# indices1 = torch.from_numpy(indices)
#print(max(indices))
# xx1=Variable(torch.zeros((5,1)))
# xx1[0] = xx[indices[0]] 
# # print(type(xx1))
# #indices = torch.permute(xx)
# print(indices)
#print(xx)




yy = xx[:,ran[0:2]]